In [ ]:
import tensorflow as tf
import os
import numpy as np
from dataset import path_to_image_crop, input_pipeline, extract_feature, images_as_float

def model1(x, y_, data_size):
    W = tf.Variable(tf.zeros([data_size, 1]))
    b = tf.Variable(tf.zeros([1]))
    y = tf.matmul(x, W) + b

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
    return tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy), y, cross_entropy

def model2(x, y_):
    def fully_connected(input, size):
        weights = tf.get_variable( 'weights', 
            shape = [input.get_shape()[1], size],
            initializer = tf.contrib.layers.xavier_initializer()
          )
        biases = tf.get_variable( 'biases',
            shape = [size],
            initializer=tf.constant_initializer(0.0)
          )
        return tf.matmul(input, weights) + biases
    def model_pass(input):
        with tf.variable_scope('hidden'):
            hidden = fully_connected(input, size = 100)
        relu_hidden = tf.nn.relu(hidden)
        with tf.variable_scope('out'):
            prediction = fully_connected(relu_hidden, size = 1)
        return prediction   
    
    predictions = model_pass(x)  
    loss = tf.reduce_mean(tf.square(predictions - y_))
    optimizer = tf.train.MomentumOptimizer(
        learning_rate = 0.01, 
        momentum = 0.9, 
        use_nesterov = True
    ).minimize(loss)
    
    return optimizer, predictions, loss

people_path = '/data/people_classification_all/fold_*_data.txt'
image_prefix = 'coarse_tilt_aligned_face'
batch_size = 32#128
image_size = 227
image_dimension = [image_size,image_size]
num_epochs = 10

model_name = "1fc_b" + str(batch_size) + "_e" + str(num_epochs - 1)
model_variable_scope = model_name

data_size = image_dimension[0] * image_dimension[1] * 3

graph = tf.Graph()
with graph.as_default():
    path_batch, label_batch = input_pipeline(people_path, batch_size, None, True)
    label_batch = extract_feature(label_batch,1)
    label_batch = tf.reshape(label_batch,[batch_size,1])

    data_batch = path_to_image_crop(path_batch, os.path.dirname(people_path), image_prefix, image_dimension)
    data_batch = tf.reshape(data_batch,[batch_size, data_size])

    x = tf.placeholder(tf.float32, [None, data_size])
    y_ = tf.placeholder(tf.float32, [None, 1])
    
    train_step, y, loss = model1(x, y_, data_size)
    #train_step, y, loss = model2(x, y_)

with tf.Session(graph = graph) as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator ()
    threads = tf.train.start_queue_runners (coord = coord)
    for i in range(num_epochs):
        batch_xs, batch_ys = session.run([data_batch, label_batch])
        batch_xs = images_as_float(batch_xs, batch_size, data_size)
        p,l,_ = session.run([y, loss, train_step], feed_dict={x: batch_xs, y_: batch_ys})
        print('%d: %s -> %s %s' % (i, l, p[i % batch_size], batch_ys[i % batch_size]))
        if (i == 0): print(batch_xs)
    coord.request_stop ()
    coord.join (threads)      